In [7]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121Note: you may need to restart the kernel to use updated packages.

INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
  Using cached https://download.pytorch.org/whl/cu121/torchvision-0.20.1%2Bcu121-cp312-cp312-win_amd64.whl (6.1 MB)
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.5.1%2Bcu121-cp312-cp312-win_amd64.whl (4.1 MB)
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.21.0
    Uninstalling torchvision-0.21.0:
      Successfully uninstalled torchvision-0.21.0
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.6.0
    Uninstalling torchaudio-2.6.0:
      Successfully uninstalled torchaudio-2.6.0



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install scikit-learn


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install datasets

   ---------------------------------------- 0.0/25.2 MB ? eta -:--:--
   -- ------------------------------------- 1.3/25.2 MB 5.6 MB/s eta 0:00:05
   --- ------------------------------------ 2.4/25.2 MB 6.1 MB/s eta 0:00:04
   ----- ---------------------------------- 3.7/25.2 MB 5.6 MB/s eta 0:00:04
   -------- ------------------------------- 5.2/25.2 MB 6.2 MB/s eta 0:00:04
   --------- ------------------------------ 6.3/25.2 MB 6.1 MB/s eta 0:00:04
   ----------- ---------------------------- 7.3/25.2 MB 6.4 MB/s eta 0:00:03
   ------------- -------------------------- 8.7/25.2 MB 5.8 MB/s eta 0:00:03
   ---------------- ----------------------- 10.2/25.2 MB 6.1 MB/s eta 0:00:03
   ------------------- -------------------- 12.6/25.2 MB 6.7 MB/s eta 0:00:02
   ---------------------- ----------------- 13.9/25.2 MB 6.7 MB/s eta 0:00:02
   ------------------------- -------------- 16.3/25.2 MB 7.1 MB/s eta 0:00:02
   ---------------------------- ----------- 17.8/25.2 MB 7.2 MB/s eta 0:00:02
 


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from datasets import load_dataset

# Load the TinyStories dataset
ds = load_dataset("roneneldan/TinyStories")

README.md:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

E:\Supporting Tools\Python\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Shreeya Kannan\.cache\huggingface\hub\datasets--roneneldan--TinyStories. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


(…)-00000-of-00004-2d5a1467fff1081b.parquet:   0%|          | 0.00/249M [00:00<?, ?B/s]

(…)-00001-of-00004-5852b56a2bd28fd9.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

(…)-00002-of-00004-a26307300439e943.parquet:   0%|          | 0.00/246M [00:00<?, ?B/s]

(…)-00003-of-00004-d243063613e5a057.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

(…)-00000-of-00001-869c898b519ad725.parquet:   0%|          | 0.00/9.99M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2119719 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21990 [00:00<?, ? examples/s]

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
import math

# Load the dataset
train_dataset = load_dataset("roneneldan/TinyStories", split="train")
valid_dataset = load_dataset("roneneldan/TinyStories", split="validation")

# Custom Tokenizer
class SimpleTokenizer:
    def __init__(self, texts, max_vocab=10000):
        self.word2idx = {"<PAD>": 0, "<UNK>": 1}
        self.idx2word = {0: "<PAD>", 1: "<UNK>"}
        self.build_vocab(texts, max_vocab)

    def build_vocab(self, texts, max_vocab):
        word_freq = {}
        for text in texts:
            for word in text.split():
                word_freq[word] = word_freq.get(word, 0) + 1
        
        sorted_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
        for word, _ in sorted_words[:max_vocab-2]:
            self.word2idx[word] = len(self.word2idx)
            self.idx2word[len(self.idx2word)] = word

    def encode(self, text, max_length):
        tokens = [self.word2idx.get(word, 1) for word in text.split()]
        return tokens[:max_length] + [0] * (max_length - len(tokens))

    def decode(self, tokens):
        return " ".join([self.idx2word.get(token, "<UNK>") for token in tokens])

# Custom Dataset
class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]['text']
        encoded = self.tokenizer.encode(text, self.max_length)
        return torch.tensor(encoded)

# Transformer Model
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_layers):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.d_model = d_model
        self.linear = nn.Linear(d_model, vocab_size)

    def forward(self, src):
        src = self.embedding(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = self.linear(output)
        return output

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return x

# Hyperparameters
MAX_LENGTH = 128
BATCH_SIZE = 32
VOCAB_SIZE = 10000
D_MODEL = 256
NHEAD = 8
NUM_LAYERS = 4
EPOCHS = 5

# Initialize tokenizer and datasets
tokenizer = SimpleTokenizer(train_dataset['text'], VOCAB_SIZE)
train_data = TextDataset(train_dataset, tokenizer, MAX_LENGTH)
valid_data = TextDataset(valid_dataset, tokenizer, MAX_LENGTH)

# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=BATCH_SIZE)

# Initialize model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerModel(VOCAB_SIZE, D_MODEL, NHEAD, NUM_LAYERS).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Training loop
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_seq = batch[:, :-1].to(device)
        target_seq = batch[:, 1:].to(device)
        
        optimizer.zero_grad()
        output = model(input_seq)
        loss = criterion(output.view(-1, VOCAB_SIZE), target_seq.contiguous().view(-1))
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {avg_loss:.4f}")

    # Validation
    model.eval()
    with torch.no_grad():
        total_val_loss = 0
        for batch in valid_loader:
            input_seq = batch[:, :-1].to(device)
            target_seq = batch[:, 1:].to(device)
            output = model(input_seq)
            loss = criterion(output.view(-1, VOCAB_SIZE), target_seq.contiguous().view(-1))
            total_val_loss += loss.item()
        avg_val_loss = total_val_loss / len(valid_loader)
        print(f"Validation Loss: {avg_val_loss:.4f}")

# Generate text
def generate_text(model, tokenizer, seed_text, max_length=50):
    model.eval()
    tokens = tokenizer.encode(seed_text, max_length)
    input_seq = torch.tensor(tokens).unsqueeze(0).to(device)
    
    with torch.no_grad():
        for _ in range(max_length):
            output = model(input_seq)
            next_token = output[0, -1, :].argmax().item()
            input_seq = torch.cat([input_seq, torch.tensor([[next_token]]).to(device)], dim=1)
            if next_token == 0:  # <PAD> token
                break
    
    return tokenizer.decode(input_seq.squeeze().tolist())

# Generate sample text
seed_text = "Once upon a time"
generated_text = generate_text(model, tokenizer, seed_text)
print(f"Generated text: {generated_text}")


E:\Supporting Tools\Python\Lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


KeyboardInterrupt: 